# 1. loading datasets

In [ ]:
import torch
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split

In [4]:


# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Get current CUDA device index (if available)
if torch.cuda.is_available():
    print("Current CUDA device index:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("No CUDA devices found.")

from torch import nn 

c:\Users\randa\OneDrive - Nanyang Technological University\Documents\GitHub\FYP\.venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


PyTorch version: 2.4.1+cpu
CUDA available: False
No CUDA devices found.


In [ ]:
file_path = './Data/12_Sep_Readings/combined_data.csv'  # Update with the correct path
cleaned_df = pd.read_csv(file_path)

In [ ]:
from common_utils import preprocess_dataset


# X = cleaned_df.drop('XY', axis=1).values this should be the rest of values
# y = cleaned_df['XY'].values this should be x and y axis values

df_train, y_train, df_test, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled, X_test_scaled = preprocess_dataset(df_train, df_test)

# X_val = X_val.astype(np.float)
# X_val = torch.FloatTensor(X_val)
# y_val = torch.LongTensor(y_val)

# 2. NN model

We will try to determine the optimal hyperparameters
1. Loss function
2. model depth and size, restricted to layer 3

In [ ]:
class MLPBuilder(nn.Module):

    def __init__(self, no_features, layers, no_labels = 64):
        super().__init__()
        layer_list = []
        
        # Input layer
        layer_list.append(nn.Linear(no_features, layers[0]))
        layer_list.append(nn.ReLU())
        layer_list.append(nn.Dropout(p=0.2))

        # Hidden layers
        for i in range(len(layers) - 1):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
            layer_list.append(nn.ReLU())
            layer_list.append(nn.Dropout(p=0.2))

        # Output layer, 2 outputs
        layer_list.append(nn.Linear(layers[-1], 2))
        layer_list.append(nn.Sigmoid())  
        # Define the MLP stack as a sequential model
        self.mlp_stack = nn.Sequential(*layer_list)
    def forward(self, x):
        logits = self.mlp_stack(x)
        return logits